In [1]:
"""
Preamble for most code and jupyter notebooks
@author: tobinsouth
@notebook date: 11 Nov 2021
"""

import numpy as np, pandas as pd, matplotlib.pyplot as plt, matplotlib as mpl, seaborn as sns
import math, string, re, pickle, json, os, sys, datetime, itertools
from collections import Counter
from tqdm import tqdm

# Set panda's options
pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 120)

# Better graphics
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('retina')
plt.style.use('seaborn-poster')

In [2]:
data = pickle.load(open('../DeepMove/data/foursquare.pk', 'rb'),  encoding='latin')

In [3]:
data.keys()

dict_keys(['data_filter', 'parameters', 'vid_lookup', 'vid_list', 'data_neural', 'uid_list'])

In [8]:
import torch
from torch.autograd import Variable

data_neural = data['data_neural']
candidate = data_neural.keys()
mode = 'train'

data_train = {}
train_idx = {}
for u in candidate:
    sessions = data_neural[u]['sessions']
    train_id = data_neural[u][mode]
    data_train[u] = {}
    for c, i in enumerate(train_id):
        trace = {}
        if mode == 'train' and c == 0:
            continue
        session = sessions[i]
        target = np.array([s[0] for s in session[1:]])

        history = []
        # if mode == 'test':
        #     test_id = data_neural[u]['train']
        #     for tt in test_id:
        #         history.extend([(s[0], s[1]) for s in sessions[tt]])
        for j in range(c):
            history.extend([(s[0], s[1]) for s in sessions[train_id[j]]])

        history_tim = [t[1] for t in history]
        history_count = [1]
        last_t = history_tim[0]
        count = 1
        for t in history_tim[1:]:
            if t == last_t:
                count += 1
            else:
                history_count[-1] = count
                history_count.append(1)
                last_t = t
                count = 1

        history_loc = np.reshape(np.array([s[0] for s in history]), (len(history), 1))
        history_tim = np.reshape(np.array([s[1] for s in history]), (len(history), 1))
        trace['history_loc'] = Variable(torch.LongTensor(history_loc))
        trace['history_tim'] = Variable(torch.LongTensor(history_tim))
        trace['history_count'] = history_count

        loc_tim = history
        loc_tim.extend([(s[0], s[1]) for s in session[:-1]])
        loc_np = np.reshape(np.array([s[0] for s in loc_tim]), (len(loc_tim), 1))
        tim_np = np.reshape(np.array([s[1] for s in loc_tim]), (len(loc_tim), 1))
        trace['loc'] = Variable(torch.LongTensor(loc_np))
        trace['tim'] = Variable(torch.LongTensor(tim_np))
        trace['target'] = Variable(torch.LongTensor(target))
        data_train[u][i] = trace
    train_idx[u] = train_id